In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import os

# =====================
# Paths
# =====================
data_path = "/kaggle/input/paddy-leaf-small/Small-80"
train_dir = os.path.join(data_path, "train")   # contains 320 images
test_dir = os.path.join(data_path, "test")     # contains 80 images

weights_path = "/kaggle/input/efficientnetv2/efficientnetv2-b0_notop.h5"  # uploaded file

# =====================
# Data Generators (with validation split)
# =====================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.25   # 25% of 320 = 80 images for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset="training"    # 240 images
)

val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset="validation"  # 80 images
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False        # 80 images
)

num_classes = train_gen.num_classes

# =====================
# Model
# =====================
try:
    base_model = EfficientNetV2B0(
        weights=weights_path,
        include_top=False,
        input_shape=(224,224,3)
    )
    print("✅ Loaded pretrained EfficientNetV2B0 weights.")
except:
    base_model = EfficientNetV2B0(
        weights=None,
        include_top=False,
        input_shape=(224,224,3)
    )
    print("⚠️ Could not load pretrained weights. Using random init.")

# Unfreeze ALL layers if weights are random
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# =====================
# Compile & Train
# =====================
model.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

callbacks = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
    ModelCheckpoint("efficientnet_with_val.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=callbacks,
    verbose=1
)

# =====================
# Evaluate on Test Set
# =====================
test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_acc*100:.2f}%")




ERROR! Session/line number was not unique in database. History logging moved to new session 29
Found 3120 images belonging to 13 classes.
Found 1040 images belonging to 13 classes.
Found 1040 images belonging to 13 classes.
✅ Loaded pretrained EfficientNetV2B0 weights.
Epoch 1/100


E0000 00:00:1756269328.529519     119 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756269328.668427     119 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


20/98 ━━━━━━━━━━━━━━━━━━━━ 29s 382ms/step - accuracy: 0.1026 - loss: 2.5987

E0000 00:00:1756269370.606459     119 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756269370.740932     119 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 731ms/step - accuracy: 0.1039 - loss: 2.5640
Epoch 1: val_accuracy improved from -inf to 0.22788, saving model to efficientnet_with_val.h5
98/98 ━━━━━━━━━━━━━━━━━━━━ 194s 991ms/step - accuracy: 0.1041 - loss: 2.5636 - val_accuracy: 0.2279 - val_loss: 2.4036 - learning_rate: 1.0000e-04
Epoch 2/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step - accuracy: 0.2274 - loss: 2.3256
Epoch 2: val_accuracy did not improve from 0.22788
98/98 ━━━━━━━━━━━━━━━━━━━━ 51s 521ms/step - accuracy: 0.2278 - loss: 2.3247 - val_accuracy: 0.1038 - val_loss: 2.5359 - learning_rate: 1.0000e-04
Epoch 3/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.3811 - loss: 1.8844
Epoch 3: val_accuracy did not improve from 0.22788
98/98 ━━━━━━━━━━━━━━━━━━━━ 51s 522ms/step - accuracy: 0.3815 - loss: 1.8833 - val_accuracy: 0.1154 - val_loss: 3.0858 - learning_rate: 1.0000e-04
Epoch 4/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.5050 - loss: 1.5004
Epoch 4: val_accuracy improve

In [ ]:
# =====================
# Plot Training Curves
# =====================
import matplotlib.pyplot as plt

# Accuracy curve
plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# Loss curve
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

# =====================
# Load Saved Model
# =====================
model = load_model("/kaggle/working/efficientnet_with_val.h5")   # or "xception_final.h5"

# =====================
# Evaluate on Test Set
# =====================
test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"✅ Loaded Model Test Accuracy: {test_acc*100:.2f}%")
print(f"✅ Loaded Model Test Loss: {test_loss:.4f}")

# =====================
# Predictions & Metrics
# =====================
y_true = test_gen.classes
y_pred_probs = model.predict(test_gen)
y_pred = np.argmax(y_pred_probs, axis=1)

# Scores
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=test_gen.class_indices.keys(),
            yticklabels=test_gen.class_indices.keys())
plt.title("Confusion Matrix - Saved Model")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()


In [ ]:
# =====================
# Model Parameters & Weights
# =====================
print("\n====================")
print(" Model Summary ")
print("====================")
model.summary()

print(f"\nTotal Parameters: {model.count_params():,}")

# Print trainable and non-trainable parameter counts
trainable_params = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_params = np.sum([K.count_params(w) for w in model.non_trainable_weights])
print(f"Trainable Params: {trainable_params:,}")
print(f"Non-trainable Params: {non_trainable_params:,}")

# Print sample weights of the first conv layer
print("\nFirst conv layer weights shape:")
print(model.layers[0].get_weights()[0].shape)

# =====================
# GFLOPs Calculation
# =====================
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

concrete_func = tf.function(lambda x: model(x))
concrete_func = concrete_func.get_concrete_function(
    tf.TensorSpec([1, 224, 224, 3], model.inputs[0].dtype)
)

frozen_func = convert_variables_to_constants_v2(concrete_func)
graph_def = frozen_func.graph.as_graph_def()

# Use TensorFlow profiler
flops = tf.profiler.experimental.get_stats_for_node_def(graph_def, "flops")
print(f"\nApproximate GFLOPs: {flops.total_float_ops / 1e9:.2f}")


In [3]:
import tensorflow as tf
import numpy as np
import os

# =====================
# Load the saved model
# =====================
# 👇 Pick the correct line depending on your model file

# Case 1: If saved as `.keras`
# model = tf.keras.models.load_model("/kaggle/input/efficientnet/other/default/1/efficientnet-new.keras")

# Case 2: If saved as `.h5`
model = tf.keras.models.load_model("/kaggle/input/eficient/other/default/1/efficientnet_with_val (1).h5")

# Case 3: If saved as TensorFlow SavedModel directory
# (must contain saved_model.pb inside the folder)
# from keras.layers import TFSMLayer
# model = TFSMLayer("/kaggle/input/efficientnet/other/default/1/efficientnet-new", call_endpoint="serving_default")

# =====================
# Parameters (Trainable + Non-trainable)
# =====================
model.summary()

# =====================
# FLOPs (GFLOPs)
# =====================
concrete = tf.function(model).get_concrete_function(
    tf.TensorSpec([1, 224, 224, 3], tf.float32)  # change 224 to your input size
)

from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

profiler = model_analyzer.Profiler(graph=concrete.graph)

opts = ProfileOptionBuilder.float_operation()
flops = profiler.profile_operations(options=opts)

total_flops = flops.total_float_ops
gflops = total_flops / 1e9

print(f"\nTotal FLOPs: {total_flops}")
print(f"GFLOPs: {gflops:.3f}")

# =====================
# Access Weights
# =====================
for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        print(f"\nLayer: {layer.name}")
        for w in weights:
            print(w.shape)


I0000 00:00:1756310316.534791      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756310316.535536      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 224, 224,  │          0 │ rescaling_4[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ normalization_4[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 112, 112,  │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 56, 56,    │      9,216 │ block1a_project_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 56, 56,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 56, 56,    │          0 │ block2a_expand_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 56, 56,    │      2,048 │ block2a_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 56, 56,    │        128 │ block2a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 56, 56,    │     36,864 │ block2a_project_… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 56, 56,    │        512 │ block2b_expand_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_act… │ (None, 56, 56,    │          0 │ block2b_expand_b

 Total params: 6,709,855 (25.60 MB)

 Trainable params: 6,649,245 (25.36 MB)

 Non-trainable params: 60,608 (236.75 KB)

 Optimizer params: 2 (12.00 B)


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th